---
title: "2023_Kronensicherung_Plesse_Analyse"
author: "Kyell Jensen"
date: "2024-08-06"
format: pdf
editor: visual
---

# 2023_Kronensicherung_Plesse_Analyse

## Kombinierte Analyse LineScale3, TreeQinetic und Versuchsaufzeichung

Nutze eine geeignete Python 3.11 Umgebung (z. B. virtuelle Environment).

## Arbeitsumgebung vorbereiten


### IMPORT: Packages

In [1]:
# Struktur & Typen
from pathlib import Path
from typing import Dict, List

# Datenverarbeitung
import json
import numpy as np
import pandas as pd
from pandas.api.types import CategoricalDtype
from slugify import slugify  # Zum Vereinheitlichen von Strings

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

# Statistik
from scipy.stats import linregress, f_oneway
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as mc

In [2]:
# Eigene Module und Funktionen
from kj_core.utils.latex_export import (
    generate_latex_table,
    generate_grouped_latex_tables,
    save_latex_table,
    build_data_dict_df
)
from kj_core.utils.labeling import (
    get_label_from_dict,
    get_color_dict
)
from kj_core import (
    CoreConfig,
    PlotManager,
    get_logger
)

# Projekteinstellungen
from project_config import (
    working_directory,
    data_export_directory,
    latex_export_directory,
    filename_clean_dataset,
    filename_clean_data_dict
)

### IMPORT: Manager Instanzen

In [3]:
logger = get_logger(__name__)
CONFIG = CoreConfig(working_directory=f"{working_directory}/combined")
PLOT_MANAGER = PlotManager(CONFIG)

2025-04-08 10:15:02 [WARNING] kj_core.core_config.set_working_directory: The directory C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined already exists.
2025-04-08 10:15:02 [INFO] kj_core.core_config.set_working_directory: Working directory set to C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined!
2025-04-08 10:15:02 [INFO] kj_core.core_config.__init__: <CoreConfig>
  Package:            kj_core
  Working Directory:  C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined
  Plot Directory:     C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\combined\plots
  Data Directory:     C:\kyellsen\005_Projekte\2024

## IMPORT: Daten Import

In [4]:
# Dateien laden
df = pd.read_feather(data_export_directory / filename_clean_dataset)

with open(data_export_directory / filename_clean_data_dict, "r", encoding="utf-8") as f:
    data_dict = json.load(f)

## ANALYSE: Explorative Datenanalyse

In [5]:
df.head(10)

id       rope_datetime treatment  release_force_target  rope_release  \
4    2 2022-03-23 11:24:23      free                   2.8       2.72310   
5    2 2022-03-23 11:24:23      free                   2.8       2.72310   
6    2 2022-03-23 11:24:23      free                   2.8       2.72310   
7    2 2022-03-23 11:24:23      free                   2.8       2.72310   
8    3 2022-03-23 11:31:12      free                   2.8       2.76205   
9    3 2022-03-23 11:31:12      free                   2.8       2.76205   
10   3 2022-03-23 11:31:12      free                   2.8       2.76205   
11   3 2022-03-23 11:31:12      free                   2.8       2.76205   
12   4 2022-03-23 11:54:33      free                   2.8       2.73950   
13   4 2022-03-23 11:54:33      free                   2.8       2.73950   

    cable_max sensor_name location   direction  height  ...  frequency_damped  \
4         NaN  Elasto(90)      StB  elongation   16.55  ...          0.441518   
5         NaN  Elasto(92)      StB  elongation    11.6  ...          0.432441   
6         NaN  Elasto(95)      StA  elongation    11.6  ...          0.422702   
7         NaN  Elasto(98)      StA  elongation   16.85  ...          0.443529   
8         NaN  Elasto(90)      StB  elongation   16.55  ...          0.440653   
9         NaN  Elasto(92)      StB  elongation    11.6  ...          0.439321   
10        NaN  Elasto(95)      StA  elongation    11.6  ...          0.436023   
11        NaN  Elasto(98)      StA  elongation   16.85  ...          0.441350   
12        NaN  Elasto(90)      StB  elongation   16.55  ...          0.439065   
13        NaN  Elasto(92)      StB  elongation    11.6  ...          0.436850   

    phase_angle    y_shift   x_shift  frequency_undamped  damping_ratio  \
4      0.200000  -5.287164  0.116641            0.442936       0.503994   
5     -0.200000 -14.233644 -0.009730            0.436601       0.873595   
6     -0.200000 -19.011309 -0.123043            0.426319       0.823807   
7     -0.065967  -1.099515  0.118608            0.445993       0.663224   
8      0.200000   7.775284  0.097887            0.441317       0.344951   
9     -0.075771   7.052210  0.094719            0.441380       0.608931   
10    -0.200000  -3.785755 -0.075741            0.438127       0.618021   
11    -0.200000 -17.747289 -0.203345            0.442888       0.525070   
12     0.200000 -23.445975  0.120929            0.439764       0.354677   
13    -0.200000 -19.454720  0.061912            0.438829       0.598701   

    metrics_warning  pearson_r     nrmse      nmae  
4             False   0.943158  0.038866  0.018011  
5             False   0.929554  0.034560  0.016183  
6             False   0.908058  0.044873  0.024274  
7             False   0.980663  0.029621  0.021087  
8             False   0.957380  0.043035  0.018619  
9             False   0.933463  0.043534  0.021255  
10            False   0.938307  0.039020  0.022498  
11            False   0.974139  0.028693  0.020715  
12            False   0.945320  0.049764  0.021509  
13            False   0.922665  0.050822  0.020659  

[10 rows x 27 columns]

In [6]:
df.columns

Index(['id', 'rope_datetime', 'treatment', 'release_force_target',
       'rope_release', 'cable_max', 'sensor_name', 'location', 'direction',
       'height', 'diameter', 'max_strain', 'max_compression', 'm_amplitude',
       'm_amplitude_2', 'initial_amplitude', 'damping_coeff',
       'frequency_damped', 'phase_angle', 'y_shift', 'x_shift',
       'frequency_undamped', 'damping_ratio', 'metrics_warning', 'pearson_r',
       'nrmse', 'nmae'],
      dtype='object')

### COMBINED: Definition von Darstellungsstandards
Festlegen von Farbcodes für einheitliche Darstellung von Sensoren und Behandlungsvarianten für alle nachfolgenden Plots.

In [7]:
color_palette = PLOT_MANAGER.color_palette_list

# Für die Spalte "treatment":
treatment_color_dict = get_color_dict(df, "treatment", PLOT_MANAGER.color_palette_list)
# Für die Spalte "sensor_name":
sensor_color_dict = get_color_dict(df, "sensor_name", PLOT_MANAGER.color_palette_list)

Index(['free', 'gefa_dynamic', 'cobra_static'], dtype='object')
Index(['Elasto(90)', 'Elasto(92)', 'Elasto(95)', 'Elasto(98)'], dtype='object')


### LATEX-EXPORT: Latex-Export von Daten für Anhang

In [8]:
variables_2 = ['id', 'sensor_name', 'treatment', 'm_amplitude', 'm_amplitude_2', 'initial_amplitude', 'damping_coeff', 'damping_ratio', 'frequency_damped', 'frequency_undamped', 'y_shift', 'pearson_r', 'nmae']

# DataFrame kopieren und die gewünschten Spalten auswählen
df_latex = df.copy()[variables_2]

# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_latex = df_latex.rename(columns={var: data_dict[var]["Zeichen"] for var in variables_2})
# Funktionsaufruf mit Beispielparametern
generate_grouped_latex_tables(
    df_latex=df_latex,
    caption="Feldversuch 2 - Ergebnisse, Schwingungsparameter vollständig",
    column_format="lrl|rrr|rr|rr|r|rr",
    group_by="treatment",
    latex_export_directory=latex_export_directory
)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_schwingungsparameter_vollstandig.tex


### PTQ: Analyse der Schwingungsparameter

In diesem Abschnitt werden die Schwingungsparameter statistisch ausgewertet. Ziel ist es, den Einfluss verschiedener Behandlungsvarianten (treatment) auf die gemessenen Schwingungsparameter zu untersuchen und dabei auch den potenziellen Einfluss der Vorspannung (rope_release) und Sensorposition (sensor_name) zu berücksichtigen.


In [9]:
variables = [
    'm_amplitude', 
    'm_amplitude_2',
    'max_strain',
    'max_compression',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    #'nrmse', 
    'nmae'
]

#### Systematischer Einfluss der Sensorposition

Ziel: Visuell erkennen, ob unterschiedliche Sensoren konsistent andere Werte liefern.

In [10]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, dodge=True)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Sensoren auf {get_label_from_dict(var, data_dict, use_titel=True)}")
    plt.xlabel("Sensorname")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show(
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_{var}", subdir="ptq_osc/sensor_only")

Ziel: Feststellen, ob die Variation durch unterschiedliche Sensorpostion relativ zur behandlungsbedingten Variation relevant ist.

In [11]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict,  dodge=True)
    # Stripplot: Punkte zur Veranschaulichung der Verteilung
    sns.stripplot(x="sensor_name", y=var, data=df, hue="treatment", palette=treatment_color_dict, dodge=True, alpha=1, jitter=True, size=5, legend=False)
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Behandlung auf {get_label_from_dict(var, data_dict, use_titel=True)} gruppiert über Sensoren")
    plt.xlabel("Sensorname")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_sensor_treatment_{var}", subdir="ptq_osc/sensor_treatment")

Ziel: Feststellen, ob die Variation durch unterschiedliche Behandlungen relativ zur sensorbedingten Variation relevant ist.

In [12]:
# Plot erstellen
for var in variables:
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name')
    sns.stripplot(x="treatment", y=var, data=df, palette=sensor_color_dict, hue='sensor_name', dodge=True, alpha=1, jitter=True, size=5, legend=False)
    plt.title(f"Einfluss der Sensoren auf {get_label_from_dict(var, data_dict, use_titel=True)} gruppiert über Behandlung")
    plt.xlabel("Behandlungsvariante")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    #plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_sensor_{var}", subdir="ptq_osc/treatment_sensor")

##### Statistische Untersuchung des Effektes der Sensorposition innerhalb einer Beobachtung

In [13]:
# 1. Deskriptive Statistiken pro Sensor
summary_by_sensor = df.groupby('sensor_name', observed=False)[variables].describe()
summary_by_sensor

m_amplitude                                                  \
                  count        mean        std     min      25%     50%   
sensor_name                                                               
Elasto(90)         27.0  251.248148  30.276998  198.30  230.025  247.60   
Elasto(92)         27.0  136.624074  27.426785   91.65  116.200  130.70   
Elasto(95)         27.0  123.061111  26.037755   81.90  105.000  117.95   
Elasto(98)         27.0  141.233333  13.683265  114.35  131.500  141.90   

                             m_amplitude_2              ... pearson_r  \
                 75%     max         count        mean  ...       75%   
sensor_name                                             ...             
Elasto(90)   275.225  309.85          27.0  130.094444  ...  0.954331   
Elasto(92)   148.300  191.30          27.0   44.124074  ...  0.929332   
Elasto(95)   136.425  173.95          27.0   52.394444  ...  0.922548   
Elasto(98)   153.200  170.85          27.0  110.111111  ...  0.975588   

                       nmae                                                    \
                  max count      mean       std       min       25%       50%   
sensor_name                                                                     
Elasto(90)   0.972302  27.0  0.023165  0.009357  0.014277  0.018246  0.021509   
Elasto(92)   0.935316  27.0  0.029160  0.006568  0.016183  0.025229  0.028271   
Elasto(95)   0.941718  27.0  0.033910  0.007446  0.022498  0.029312  0.032552   
Elasto(98)   0.987644  27.0  0.030302  0.009289  0.019387  0.021929  0.028730   

                                 
                  75%       max  
sensor_name                      
Elasto(90)   0.025602  0.062792  
Elasto(92)   0.033684  0.041554  
Elasto(95)   0.037804  0.050641  
Elasto(98)   0.036272  0.054711  

[4 rows x 88 columns]

In [14]:
# 3. ANOVA: Teste, ob die Unterschiede zwischen den Sensoren signifikant sind
print("\nANOVA-Tests für Unterschiede zwischen den Sensoren:")
for var in variables:
    # Fit eines linearen Modells mit sensor_name als Prädiktor
    model = smf.ols(formula=f"{var} ~ C(sensor_name)", data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(f"\nANOVA für {var}:")
    print(anova_table)


ANOVA-Tests für Unterschiede zwischen den Sensoren:

ANOVA für m_amplitude:
                       sum_sq     df           F        PR(>F)
C(sensor_name)  284912.661574    3.0  149.907389  1.270263e-37
Residual         65887.160926  104.0         NaN           NaN

ANOVA für m_amplitude_2:
                       sum_sq     df          F        PR(>F)
C(sensor_name)  145674.651736    3.0  79.526795  8.219855e-27
Residual         63501.296852  104.0        NaN           NaN

ANOVA für max_strain:
                       sum_sq     df           F        PR(>F)
C(sensor_name)  492818.234352    3.0  180.397796  4.550338e-41
Residual         94703.848148  104.0         NaN           NaN

ANOVA für max_compression:
                       sum_sq     df          F        PR(>F)
C(sensor_name)  206732.324722    3.0  50.247393  3.723270e-20
Residual        142628.704444  104.0        NaN           NaN

ANOVA für initial_amplitude:
                      sum_sq     df         F   PR(>F)
C(sensor_na

##### Optimaler Ansatz: Versuche Mixed-Linear Model

Die Daten sind hierarchisch: mehrere Messungen (vier Sensoren) pro Beobachtungseinheit (`id`). Ein Mixed-Effects Modell könnte diese Struktur abbilden, indem zufällige Effekte für `id` und feste Effekte für `treatment` sowie `sensor_name` berücksichtigt werden. Zusätzlich kann `rope_release` als Kovariate eingeführt werden. Dieser Ansatz wären theoretisch optimal, aber aufgrund der geringen Stichprobengröße und der komplexen Datenstruktur treten Konvergenzprobleme auf.

In [15]:
# Ergebnisse als Dictionary speichern
results_dict = {}

for var in variables:
    model = smf.mixedlm(
        f"{var} ~ C(treatment) + C(sensor_name) + rope_release", 
        data=df, 
        groups=df["id"]
    )
    result = model.fit()
    print(f"\n### Ergebnisse für {var}:")
    print(result.summary())
    print("\n")

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



### Ergebnisse für m_amplitude:
                    Mixed Linear Model Regression Results
Model:                   MixedLM        Dependent Variable:        m_amplitude
No. Observations:        108            Method:                    REML       
No. Groups:              27             Scale:                     133.1670   
Min. group size:         4              Log-Likelihood:            -401.1554  
Max. group size:         4              Converged:                 Yes        
Mean group size:         4.0                                                  
------------------------------------------------------------------------------
                              Coef.   Std.Err.    z    P>|z|  [0.025   0.975] 
------------------------------------------------------------------------------
Intercept                     176.658    8.767  20.150 0.000  159.475  193.842
C(treatment)[T.gefa_dynamic]  -22.842    2.722  -8.392 0.000  -28.177  -17.507
C(treatment)[T.cobra_static]  -44.541   

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed t


### Ergebnisse für m_amplitude_2:
                   Mixed Linear Model Regression Results
Model:                  MixedLM      Dependent Variable:      m_amplitude_2
No. Observations:       108          Method:                  REML         
No. Groups:             27           Scale:                   180.6924     
Min. group size:        4            Log-Likelihood:          -416.5641    
Max. group size:        4            Converged:               No           
Mean group size:        4.0                                                
---------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------
Intercept                    126.698   10.211  12.408 0.000 106.685 146.711
C(treatment)[T.gefa_dynamic] -22.014    3.170  -6.944 0.000 -28.227 -15.801
C(treatment)[T.cobra_static] -49.446    3.170 -15.600 0.000 -55.659 -43.

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(


...

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)



### Ergebnisse für frequency_damped:
                 Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    frequency_damped
No. Observations:    108        Method:                REML            
No. Groups:          27         Scale:                 0.0010          
Min. group size:     4          Log-Likelihood:        196.7684        
Max. group size:     4          Converged:             Yes             
Mean group size:     4.0                                               
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                     0.440    0.024 18.680 0.000  0.394  0.486
C(treatment)[T.gefa_dynamic]  0.013    0.007  1.826 0.068 -0.001  0.028
C(treatment)[T.cobra_static]  0.046    0.007  6.287 0.000  0.032  0.060
C(sensor_name)[T.Elasto(92)] -0.044    0.00

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed t


### Ergebnisse für frequency_undamped:
                 Mixed Linear Model Regression Results
Model:               MixedLM   Dependent Variable:   frequency_undamped
No. Observations:    108       Method:               REML              
No. Groups:          27        Scale:                0.0007            
Min. group size:     4         Log-Likelihood:       213.8221          
Max. group size:     4         Converged:            No                
Mean group size:     4.0                                               
-----------------------------------------------------------------------
                             Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------
Intercept                     0.437    0.020 21.963 0.000  0.398  0.476
C(treatment)[T.gefa_dynamic]  0.015    0.006  2.464 0.014  0.003  0.027
C(treatment)[T.cobra_static]  0.060    0.006  9.792 0.000  0.048  0.073
C(sensor_name)[T.Elasto(92)] -0.033    0.

C:\kyellsen\006_Packages\venv_Python311\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


Um dennoch aussagekräftige Ergebnisse zu erhalten, wird ein zweistufiges Verfahren genutzt.
Zwei Varianten:
- Mittlung über alle Sensoren innerhalb eine Beobachtung. Die Variation zwischen den Sensoren bleibt unberücksichtigt und verfälscht ggf. die Ergebnisse. Vorteil: Sehr einfach und stabil
- Werte um Sensor-Effekt manuell adjustieren und dann aggregieren wie zuvor. Vorteile: Sensoren verfälschen nicht die Ergebnisse. Nachteil ist die höhere Komplexität und das die Gesamtvariation der Modell am ende verfälsch ist.

##### Alternative 1: Vereinfachtes Vorgehen durch Aggregation (Mittelwert)

In [16]:
df_mean = df.groupby("id").agg({var: "mean" for var in variables}).reset_index()
# Benenne die Spalten um, damit klar ist, dass es sich um Mittelwerte handelt.
df_mean = df_mean.rename(columns={var: f"{var}_mean" for var in variables})

# Füge zusätzliche Variablen (z.B. treatment und rope_release) hinzu.
treatment_df = df[["id", "treatment", "rope_release"]].drop_duplicates()
df_mean = df_mean.merge(treatment_df, on="id")
df_mean.describe()

id  m_amplitude_mean  m_amplitude_2_mean  max_strain_mean  \
count  27.000000         27.000000           27.000000        27.000000   
mean   15.000000        163.041667           84.181019       248.391667   
std     7.937254         22.751519           21.178032        25.407331   
min     2.000000        124.700000           50.825000       204.125000   
25%     8.500000        148.368750           61.868750       232.437500   
50%    15.000000        162.850000           87.362500       244.750000   
75%    21.500000        181.712500          102.618750       262.400000   
max    28.000000        204.950000          115.737500       297.150000   

       max_compression_mean  initial_amplitude_mean  damping_coeff_mean  \
count             27.000000               27.000000           27.000000   
mean             -93.480556              169.790250            0.434036   
std               31.077756                5.462118            0.200727   
min             -144.950000              164.414861            0.218833   
25%             -125.262500              166.161477            0.276357   
50%              -80.800000              169.854061            0.336295   
75%              -73.425000              170.241598            0.692071   
max              -43.775000              189.345841            0.748626   

       damping_ratio_mean  frequency_damped_mean  frequency_undamped_mean  \
count           27.000000              27.000000                27.000000   
mean             0.968812               0.456238                 0.463587   
std              0.442173               0.022212                 0.027848   
min              0.499237               0.435009                 0.436606   
25%              0.636314               0.437784                 0.439283   
50%              0.748273               0.450018                 0.453386   
75%              1.469274               0.466431                 0.483996   
max              1.757161               0.508053                 0.522475   

       pearson_r_mean  nmae_mean  rope_release  
count       27.000000  27.000000     27.000000  
mean         0.888360   0.029134      2.371609  
std          0.053055   0.006646      0.320477  
min          0.799567   0.019889      1.946800  
25%          0.842485   0.024730      2.012975  
50%          0.894191   0.027968      2.356550  
75%          0.946130   0.033468      2.748225  
max          0.956986   0.043212      2.780700

##### Alternative 2: Werte um Sensor-Effekt und Vorspannung Adjustieren und dann Aggregieren (Mittelwert)

Nicht jeder Parameter wird durch `rope_release` beeinflusst. Nur für jene Parameter, bei denen ein signifikanter Einfluss von `rope_release` festgestellt wird, soll dieser Effekt herausgerechnet werden. Auf diese Weise entstehen "bereinigte" Werte, in denen der lineare Einfluss von `rope_release` entfernt ist.

In [17]:
# Definiere die Liste der Variablen, bei denen rope_release entfernt werden soll
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']
model_dict = {}
# Erstelle eine Kopie des Original-DataFrames
df_copy = df.copy()

for var in variables:
    try:
        # Wähle die Modellformel abhängig davon, ob rope_release berücksichtigt werden soll
        if var in relevant_vars_rope_release:
            formula = f"{var} ~ C(sensor_name) + rope_release"
        else:
            formula = f"{var} ~ C(sensor_name)"
            
        model = smf.ols(formula, data=df).fit()
        
        # Speichere das Modell im Dictionary (so hast du per Variable die komplette Summary)
        model_dict[var] = model
        # Berechne adjustierte Werte:
        # y_adj = y - (fitted effect - globaler Intercept)
        df_copy[f"{var}_adj"] = df_copy[var] - (model.fittedvalues - model.params['Intercept'])
    except Exception as e:
        print(f"Fehler bei {var}: {e}")

# Aggregiere über die ID – hier wird der Mittelwert der adjustierten Werte berechnet.
df_adj = df_copy.groupby("id").agg({f"{var}_adj": "mean" for var in variables}).reset_index()
# Ergänze zusätzliche Variablen (z.B. treatment und rope_release)
treatment_df = df[["id", "treatment", "rope_release"]].drop_duplicates()
df_adj = df_adj.merge(treatment_df, on="id")

# Ausgabe der deskriptiven Statistik
print(df_adj.describe())


              id  m_amplitude_adj  m_amplitude_2_adj  max_strain_adj  \
count  27.000000        27.000000          27.000000       27.000000   
mean   15.000000       157.368958         106.177715      217.418613   
std     7.937254        18.886461          20.929978       16.410941   
min     2.000000       126.721101          73.696181      186.504963   
25%     8.500000       138.251643          83.173218      206.941647   
50%    15.000000       157.684238         109.601825      214.816110   
75%    21.500000       177.379976         127.421895      232.252152   
max    28.000000       183.821853         134.024166      242.458537   

       max_compression_adj  initial_amplitude_adj  damping_coeff_adj  \
count            27.000000              27.000000          27.000000   
mean           -117.547632             175.130475           0.267204   
std              30.785344               5.462118           0.200727   
min            -168.905719             169.755086           0.0

##### Zusammenführen beider Alternativen für Vergleich

In [18]:
# --- Zusammenführen beider Ansätze ---
# Beide DataFrames enthalten nun jeweils den Mittelwert pro ID:
# - df_mean: direkte Mittelwerte mit Suffix _mean
# - df_adj: sensor-adjustierte Mittelwerte mit Suffix _adj
grouped_df = pd.merge(df_mean, df_adj, on=["id", "treatment", "rope_release"], how="inner")
grouped_df

id  m_amplitude_mean  m_amplitude_2_mean  max_strain_mean  \
0    2          198.6125            103.9750          293.625   
1    3          204.9500            115.1000          293.800   
2    4          199.6500            115.7375          272.650   
3    5          188.4000            108.7875          267.575   
4    6          186.4250            110.7250          229.975   
5    7          188.6250            111.0500          251.300   
6    8          168.9625            102.7000          215.500   
7    9          165.8375            102.5375          221.200   
8   10          163.3000            100.0625          209.275   
9   11          177.0625             94.5000          247.825   
10  12          182.6750             89.2875          287.200   
11  13          180.7500             88.7625          297.150   
12  14          152.6250             80.5125          241.900   
13  15          162.8500             88.9625          238.350   
14  16          166.3875             87.3625          258.000   
15  17          149.7375             84.3000          243.150   
16  18          151.7625             86.6625          254.725   
17  19          143.1375             74.3875          244.750   
18  20          155.5250             61.7625          240.800   
19  21          153.9875             66.2750          247.975   
20  22          148.5875             55.8250          266.800   
21  23          146.6125             60.8625          242.050   
22  24          138.6750             56.9875          234.900   
23  25          148.1500             61.9750          253.650   
24  26          128.2125             55.9250          228.650   
25  27          125.9250             50.8250          219.675   
26  28          124.7000             57.0375          204.125   

    max_compression_mean  initial_amplitude_mean  damping_coeff_mean  \
0               -124.350              169.960362            0.310671   
1               -128.225              170.248463            0.230183   
2               -136.150              170.814318            0.218833   
3               -116.775              170.279813            0.284570   
4               -144.950              184.280012            0.228966   
5               -144.325              166.418673            0.230392   
6               -126.325              170.546205            0.226999   
7               -126.175              170.234733            0.268143   
8               -130.575              170.351945            0.235773   
9               -120.625              189.345841            0.351440   
10               -95.150              169.854061            0.330830   
11               -79.050              169.852400            0.386842   
12               -79.375              164.822827            0.382343   
13               -91.525              170.059247            0.318310   
14               -83.050              169.931093            0.336295   
15               -79.275              170.044870            0.331537   
16               -68.875              169.952585            0.331128   
17               -71.050              169.761127            0.380151   
18               -76.150              164.726630            0.702507   
19               -80.800              164.950203            0.668239   
20               -43.775              165.904281            0.707464   
21               -75.800              169.177693            0.721738   
22               -48.550              164.739936            0.690053   
23               -78.600              164.759650            0.694089   
24               -62.375              169.407241            0.698741   
25               -54.300              169.497670            0.748626   
26               -57.800              164.414861            0.704121   

    damping_ratio_mean  frequency_damped_mean  frequency_undamped_mean  ...  \
0             0.716155               0.435047                 0.437962  ...   
1       

Plotten und Vergleich beider Ergebnisse

In [19]:
for var in variables:
    # Transformiere die Daten in das Long-Format: 
    # Es werden die beiden Spalten f"{var}_mean" und f"{var}_adj" unter einer neuen Spalte "aggregation" zusammengeführt,
    # und die Werte in der Spalte "value" gespeichert.
    df_long = pd.melt(
        grouped_df,
        id_vars=["id", "treatment", "rope_release"],
        value_vars=[f"{var}_mean", f"{var}_adj"],
        var_name="aggregation",
        value_name="value"
    )
    # Kürze die Beschriftungen, sodass nur "mean" bzw. "adj" übrig bleiben
    df_long["aggregation"] = df_long["aggregation"].str.replace(f"{var}_", "", regex=False)
    
    # Erstelle den Plot: Boxplots nach Behandlung, getrennt nach Aggregationsmethode
    fig = plt.figure(figsize=(8, 5))
    sns.boxplot(x="treatment", y="value", hue="aggregation", data=df_long, dodge=True)
    
    # Titel und Achsentitel setzen
    plt.title(f"Einfluss der Behandlung auf {get_label_from_dict(var, data_dict, use_titel=True)}")
    plt.xlabel("Behandlung")
    plt.ylabel(get_label_from_dict(var, data_dict, use_full=True))
    plt.tight_layout()
    
    # Speichere den Plot (ersetze PLOT_MANAGER.save_plot durch plt.show() oder deine eigene Funktion, falls nötig)
    PLOT_MANAGER.save_plot(fig, filename=f"effect_treatment_{var}", subdir="ptq_osc/treatment_comparison")


In [20]:
def extract_model_info(model):
    """
    Extrahiert zentrale Kennzahlen aus einem OLS-Modell.
    Liefert ein Dictionary mit R-squared, Adj. R-squared, sowie den Koeffizienten,
    Standardfehler, t-Werten und p-Werten für alle Parameter.
    """
    info = {
        "R²": model.rsquared,
        "adj_R²": model.rsquared_adj,
    }
    # Iteriere über alle Parameter im Modell
    for param in model.params.index:
        info[f"coef_{param}"] = model.params[param]
        info[f"std_err_{param}"] = model.bse[param]
        info[f"t_{param}"] = model.tvalues[param]
        info[f"p_{param}"] = model.pvalues[param]
    return info

In [21]:
# Erstelle eine Liste, in der pro Variable das extrahierte Dictionary gespeichert wird.
model_info_list = []
for var, model in model_dict.items():
    info = extract_model_info(model)
    info['variable'] = var  # Variable als Kennung hinzufügen
    model_info_list.append(info)

# Konvertiere die Liste in einen DataFrame und setze "variable" als Index
df_model_info = pd.DataFrame(model_info_list).set_index("variable")
df_model_info.index.name = "Statistik"
df_model_info = df_model_info.T

In [22]:
def format_value(x):
    # Prüfe, ob der Wert numerisch ist
    if isinstance(x, (int, float, np.number)):
        if abs(x) < 0.0001:
            return f"{x:.2e}"
        else:
            return f"{x:.2f}"
    return x

# Wende die Funktion auf alle Zellen von df_model_info an
df_model_info = df_model_info.applymap(format_value)
df_model_info

C:\Users\mail\AppData\Local\Temp\ipykernel_26076\3004503431.py:11: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_model_info = df_model_info.applymap(format_value)


Statistik                            m_amplitude m_amplitude_2 max_strain  \
R²                                          0.86          0.70       0.91   
adj_R²                                      0.85          0.69       0.90   
coef_Intercept                            157.37        106.18     217.42   
std_err_Intercept                          16.40         18.48      17.44   
t_Intercept                                 9.60          5.75      12.47   
p_Intercept                             5.86e-16      9.38e-08   2.67e-22   
coef_C(sensor_name)[T.Elasto(92)]        -114.62        -85.97    -128.81   
std_err_C(sensor_name)[T.Elasto(92)]        5.95          6.70       6.32   
t_C(sensor_name)[T.Elasto(92)]            -19.28        -12.83     -20.38   
p_C(sensor_name)[T.Elasto(92)]          5.91e-36      4.35e-23   6.43e-38   
coef_C(sensor_name)[T.Elasto(95)]        -128.19        -77.70    -142.29   
std_err_C(sensor_name)[T.Elasto(95)]        5.95          6.70       6.32   
t_C(sensor_name)[T.Elasto(95)]            -21.56        -11.60     -22.51   
p_C(sensor_name)[T.Elasto(95)]          5.68e-40      2.13e-20   1.47e-41   
coef_C(sensor_name)[T.Elasto(98)]        -110.01        -19.98    -179.16   
std_err_C(sensor_name)[T.Elasto(98)]        5.95          6.70       6.32   
t_C(sensor_name)[T.Elasto(98)]            -18.50         -2.98     -28.34   
p_C(sensor_name)[T.Elasto(98)]          1.59e-34          0.00   1.94e-50   
coef_rope_release                          39.58         10.08      60.52   
std_err_rope_release                        6.68          7.53       7.11   
t_rope_release                              5.92          1.34       8.52   
p_rope_release                          4.23e-08          0.18   1.45e-13   

Statistik                            max_compression initial_amplitude  \
R²                                              0.60              0.09   
adj_R²                                          0.58              0.06   
coef_Intercept                               -117.55            175.13   
std_err_Intercept                              27.75              2.05   
t_Intercept                                    -4.24             85.27   
p_Intercept                                 4.97e-05          4.54e-98   
coef_C(sensor_name)[T.Elasto(92)]              98.13             -8.36   
std_err_C(sensor_name)[T.Elasto(92)]           10.06              2.90   
t_C(sensor_name)[T.Elasto(92)]                  9.75             -2.88   
p_C(sensor_name)[T.Elasto(92)]              2.61e-16              0.00   
coef_C(sensor_name)[T.Elasto(95)]              98.59             -7.71   
std_err_C(sensor_name)[T.Elasto(95)]           10.06              2.90   
t_C(sensor_name)[T.Elasto(95)]                  9.80             -2.65   
p_C(sensor_name)[T.Elasto(95)]              2.07e-16              0.01   
coef_C(sensor_name)[T.Elasto(98)]              25.45             -5.29   
std_err_C(sensor_name)[T.Elasto(98)]           10.06              2.90   
t_C(sensor_name)[T.Elasto(98)]                  2.53             -1.82   
p_C(sensor_name)[T.Elasto(98)]                  0.01              0.07   
coef_rope_release                             -13.27               nan   
std_err_rope_release                           11.31               nan   
t_rope_release                                 -1.17               nan   
p_rope_release                                  0.24               nan   

Statistik                            damping_coeff damping_ratio  \
R²                                            0.30          0.34   
adj_R²                                        0.28          0.32   
coef_Intercept                                0.27          0.55   
std_err_Intercept                             0.04          0.11   
t_Intercept                                   6.03          5.26   
p_Intercept                               2.49e-08      7.60e-07   
coef_C(sensor_name)[T.Elasto(92)]             0.3

In [23]:
# Manuelles Escapen der Unterstriche im Index
df_model_info.index = df_model_info.index.astype(str).str.replace('(sensor_name)[T.', r'(sensor)[')
df_model_info.index = df_model_info.index.astype(str).str.replace('_', r'\_')


# Spaltennamen mit den Kurzbezeichnungen (Zeichen) aus dem data_dict umbenennen
df_model_info = df_model_info.rename(columns={var: data_dict[var]["Zeichen"] for var in variables})

column_format = "l" + "r" * (len(df_model_info.columns))
latex_string = df_model_info.to_latex(index=True, escape=False, float_format="%.2f", column_format=column_format)

# Definiere Beschriftung und lange Beschriftung (Caption)
caption = "Feldversuch 2: Ergebnisse, Zusammenfassung der OLS-Modelle"
caption_long = "Übersicht über zentrale Kennzahlen der OLS-Modelle: R², Adjusted R², Koeffizienten, Standardfehler, t-Werte und p-Werte für alle Parameter."

# Funktion zum Speichern der LaTeX-Tabelle (du passt diese Funktion ggf. an deine Gegebenheiten an)
save_latex_table(latex_string, caption, latex_export_directory, caption_long)

Content saved to: C:\kyellsen\005_Projekte\2024_BA\032_Feldversuch_2023_Plesse\030_Analysen\2023_Kronensicherung_Plesse_Kraefte_Schwingungen\working_directory\export_latex\feldversuch_2_ergebnisse_zusammenfassung_der_ols_modelle.tex


Auf dieser Basis können einfache OLS-Modelle geschätzt werden, z. B. `Parameter ~ C(treatment)`. Diese Modelle sind einfacher und sollten stabil konvergieren.

#### Systematischer Einfluss der Behandlungsvariante

In [24]:
grouped_df

id  m_amplitude_mean  m_amplitude_2_mean  max_strain_mean  \
0    2          198.6125            103.9750          293.625   
1    3          204.9500            115.1000          293.800   
2    4          199.6500            115.7375          272.650   
3    5          188.4000            108.7875          267.575   
4    6          186.4250            110.7250          229.975   
5    7          188.6250            111.0500          251.300   
6    8          168.9625            102.7000          215.500   
7    9          165.8375            102.5375          221.200   
8   10          163.3000            100.0625          209.275   
9   11          177.0625             94.5000          247.825   
10  12          182.6750             89.2875          287.200   
11  13          180.7500             88.7625          297.150   
12  14          152.6250             80.5125          241.900   
13  15          162.8500             88.9625          238.350   
14  16          166.3875             87.3625          258.000   
15  17          149.7375             84.3000          243.150   
16  18          151.7625             86.6625          254.725   
17  19          143.1375             74.3875          244.750   
18  20          155.5250             61.7625          240.800   
19  21          153.9875             66.2750          247.975   
20  22          148.5875             55.8250          266.800   
21  23          146.6125             60.8625          242.050   
22  24          138.6750             56.9875          234.900   
23  25          148.1500             61.9750          253.650   
24  26          128.2125             55.9250          228.650   
25  27          125.9250             50.8250          219.675   
26  28          124.7000             57.0375          204.125   

    max_compression_mean  initial_amplitude_mean  damping_coeff_mean  \
0               -124.350              169.960362            0.310671   
1               -128.225              170.248463            0.230183   
2               -136.150              170.814318            0.218833   
3               -116.775              170.279813            0.284570   
4               -144.950              184.280012            0.228966   
5               -144.325              166.418673            0.230392   
6               -126.325              170.546205            0.226999   
7               -126.175              170.234733            0.268143   
8               -130.575              170.351945            0.235773   
9               -120.625              189.345841            0.351440   
10               -95.150              169.854061            0.330830   
11               -79.050              169.852400            0.386842   
12               -79.375              164.822827            0.382343   
13               -91.525              170.059247            0.318310   
14               -83.050              169.931093            0.336295   
15               -79.275              170.044870            0.331537   
16               -68.875              169.952585            0.331128   
17               -71.050              169.761127            0.380151   
18               -76.150              164.726630            0.702507   
19               -80.800              164.950203            0.668239   
20               -43.775              165.904281            0.707464   
21               -75.800              169.177693            0.721738   
22               -48.550              164.739936            0.690053   
23               -78.600              164.759650            0.694089   
24               -62.375              169.407241            0.698741   
25               -54.300              169.497670            0.748626   
26               -57.800              164.414861            0.704121   

    damping_ratio_mean  frequency_damped_mean  frequency_undamped_mean  ...  \
0             0.716155               0.435047                 0.437962  ...   
1       

In [25]:
df_adj.columns = df_adj.columns.str.replace('_adj$', '', regex=True)
desired_order = ['id', 'treatment', 'rope_release']
other_columns = [col for col in df_adj.columns if col not in desired_order]
new_order = desired_order + other_columns
df_adj = df_adj[new_order]
df_adj

id     treatment  rope_release  m_amplitude  m_amplitude_2  max_strain  \
0    2          free       2.72310   179.026173     122.427053  241.378667   
1    3          free       2.76205   183.821853     133.159258  239.196295   
2    4          free       2.73950   179.414486     134.024166  219.411089   
3    5          free       2.36475   182.998813     130.853370  237.017089   
4    6          free       2.38000   180.420148     132.637079  198.494113   
5    7          free       2.36920   183.047661     133.070993  220.472762   
6    8          free       1.96440   179.409005     128.803239  209.172478   
7    9          free       1.97830   175.733779     128.500563  214.031208   
8   10          free       1.94680   174.443194     126.343228  204.012683   
9   11  gefa_dynamic       2.75785   156.100609     112.601614  193.475492   
10  12  gefa_dynamic       2.78010   160.832352     107.164731  231.503855   
11  13  gefa_dynamic       2.76350   159.564456     106.807136  242.458537   
12  14  gefa_dynamic       2.30735   149.495968     103.157226  214.816110   
13  15  gefa_dynamic       2.35655   157.773406     111.111063  208.288378   
14  16  gefa_dynamic       2.34755   161.667168     109.601825  228.483085   
15  17  gefa_dynamic       2.02755   157.684238     109.766397  233.000450   
16  18  gefa_dynamic       2.08095   157.595420     111.590379  241.343521   
17  19  gefa_dynamic       1.99740   152.277713     100.157947  236.425219   
18  20  cobra_static       2.75695   134.598735      79.873190  186.504963   
19  21  cobra_static       2.76580   132.710911      84.296441  193.144334   
20  22  cobra_static       2.78070   126.721101      73.696181  211.067541   
21  23  cobra_static       2.34425   142.022797      83.135104  212.732811   
22  24  cobra_static       2.34405   134.093214      79.262121  205.594916   
23  25  cobra_static       2.45465   139.190158      83.134264  217.651070   
24  26  cobra_static       1.99840   137.313129      81.685363  220.264696   
25  27  cobra_static       1.98435   135.581792      76.727051  212.140044   
26  28  cobra_static       1.95740   135.423597      83.211331  198.221139   

    max_compression  initial_amplitude  damping_coeff  damping_ratio  \
0       -143.752300         175.300587       0.143838       0.300369   
1       -147.110378         175.588688       0.063350       0.108458   
2       -155.334649         176.154543       0.052001       0.083451   
3       -140.933108         175.620039       0.117738       0.239288   
4       -168.905719         189.620237       0.062134       0.108539   
5       -168.424050         171.758898       0.063559       0.111597   
6       -155.796316         175.886430       0.060166       0.106862   
7       -155.461843         175.574958       0.101311       0.201769   
8       -160.279892         175.692170       0.068940       0.126455   
9       -139.566118         194.686066       0.184608       0.357757   
10      -113.795829         175.194286       0.163997       0.312770   
11       -97.916135         175.192625       0.220009       0.438106   
12      -104.294887         170.163052       0.215510       0.452767   
13      -115.791934         175.399472       0.151478       0.290829   
14      -107.436376         175.271318       0.169462       0.332488   
15      -107.908226         175.385095       0.164705       0.322501   
16       -96.799533         175.292810       0.164296       0.320914   
17      -100.083359         175.101352       0.213319       0.437993   
18       -95.103062         170.066855       0.535674       1.211436   
19       -99.635611         170.290428       0.501407       0.956138   
20       -62.412867         171.244506       0.540631       1.287027   
21      -100.230172         174.517918       0.554905       1.073340   
22       -72.982826         170.080161       0.523220       1.046423   
23      -101.565009         170.099875       0.527256       1.097952   
24       -91.395088

In [26]:
import pandas as pd
from tabulate import tabulate
import re

# Liste der Variablen, die in die Tabelle übernommen werden sollen
variables_spec = [
    'm_amplitude_2',
    'max_compression',
    'damping_coeff',
    'damping_ratio',
    'frequency_damped',
    'frequency_undamped',
]

# Umbenennung der Spalten für LaTeX-Notation
column_rename_map = {
    'm_amplitude_2': r'$mA_2$',
    'max_compression': r'$\text{max\_C}$',
    'damping_coeff': r'$\delta$',
    'damping_ratio': r'$D$',
    'frequency_damped': r'$f_d$',
    'frequency_undamped': r'$f_0$',
}

# Funktion zur Erstellung der Modellgüte-Kennzahlen für alle Variablen
def create_model_metrics_table(variables, results):
    metrics_data = {
        "Kennzahl": ["R²", "Adj. R²", "F-St.", "AIC", "N"]
    }

    for var in variables:
        col_name = column_rename_map.get(var, var)  # Verwende gekürzten Namen, falls vorhanden
        if var not in results or 'error' in results[var]:
            metrics_data[col_name] = ["n/a"] * 5
        else:
            model = results[var]['model']
            metrics_data[col_name] = [
                f"{model.rsquared:.4f}",
                f"{model.rsquared_adj:.4f}",
                f"{model.fvalue:.4f}",
                f"{model.aic:.4f}",
                f"{model.nobs:.0f}"
            ]

    # Erstelle die Tabelle mit tabulate
    metrics_df = pd.DataFrame(metrics_data)
    return tabulate(
        metrics_df,
        headers="keys",
        tablefmt="latex_raw",
        floatfmt=".4f",
        showindex=False)

# Funktion zur Erstellung einer LaTeX-Tabelle für die Koeffizienten einer Variable
def create_latex_table_for_variable(var, results):
    if var not in results:
        return f"%% Keine Ergebnisse für Variable {var} vorhanden."
    
    model_result = results[var]
    if 'error' in model_result:
        return f"%% Fehler beim Anpassen des Modells für {var}: {model_result['error']}"
    
    model = model_result['model']
    summary = model.summary2().tables[1]  # Zugriff auf die Tabelle der Koeffizienten

    # Erstelle eine LaTeX-Tabelle mit tabulate für die Koeffizienten
    latex_table = tabulate(
        summary,
        headers=summary.columns,
        tablefmt="latex_booktabs",
        floatfmt=".4f",
        showindex=True
    )

    # Escape Unterstriche in der Variable für LaTeX
    escaped_var = re.sub(r'_', r'\_', var)
    shortened_var = column_rename_map.get(var, escaped_var)

    # Füge die LaTeX-Caption zur Tabelle hinzu
    return f"""
\\begin{{table}}[ht]
    \\centering
    \\caption{{Modellzusammenfassung für {escaped_var} ({shortened_var})}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {latex_table}
    \\end{{adjustbox}}
\\end{{table}}
"""

# Erstelle die Modellgüte-Tabelle für alle Variablen
latex_metrics_table = create_model_metrics_table(variables_spec, results)
print("""
\\begin{table}[ht]
    \\centering
    \\caption{Modellgüte für alle Variablen}
    \\begin{adjustbox}{max width=\\textwidth}
""")
print(latex_metrics_table)
print("""
    \\end{adjustbox}
\\end{table}

\\vspace{1cm}
""")

# Erstelle und print die LaTeX-Tabellen für die Koeffizienten jeder Variable
for var in variables_spec:
    latex_output = create_latex_table_for_variable(var, results)
    print(latex_output)

NameError: name 'results' is not defined

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables), 2, (len(variables) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_with_rope_release", subdir="combined")


Für die betroffenen Parameter wird der Einfluss von `rope_release` mithilfe der bereits angepassten Modelle (`Parameter ~ C(treatment) + rope_release`) entfernt. Dazu werden Vorhersagen für einen konstanten `rope_release`-Wert (den Mittelwert) berechnet und mit den tatsächlichen Werten verglichen. Die daraus resultierenden bereinigten Werte sind frei von Variation, die auf `rope_release` zurückzuführen wäre.

In [ ]:
grouped_adjusted_df = grouped_df.copy()

# Bereinigung: Wir setzen rope_release auf seinen Mittelwert
rope_mean = grouped_df["rope_release"].mean()

for var in relevant_vars_rope_release:
    # Zugehöriges Modellobjekt abrufen
    model = results[var]["model"]

    # Vorhersage mit tatsächlichen rope_release-Werten
    predicted_current = model.predict(grouped_df)

    # Vorhersage, wenn rope_release = rope_mean gesetzt wird
    df_mean_rope = grouped_df.copy()
    df_mean_rope["rope_release"] = rope_mean
    predicted_mean = model.predict(df_mean_rope)

    # Angepasste Werte berechnen:
    actual = grouped_df[var].values
    adjusted = actual + (predicted_mean - predicted_current)
    grouped_adjusted_df[var] = adjusted

In [ ]:
grouped_df.describe()

In [ ]:
grouped_adjusted_df.describe()

##### Latex Tabelle Output


In [ ]:
data_dict

In [ ]:
# DataFrame mit den relevanten Spalten erstellen
variables_latex = [
    'm_amplitude', 
    'm_amplitude_2',
    'initial_amplitude',
    'damping_coeff', 
    'damping_ratio', 
    'frequency_damped', 
    'frequency_undamped',
    'pearson_r',
    'nmae'
]
df_latex = grouped_adjusted_df[variables_latex + ['treatment']].copy()
df_latex.columns

In [ ]:


# Spaltennamen entsprechend der LaTeX-Notation umbenennen
df_latex.rename(columns=data_dict, inplace=True)

# Erstellung deskriptiver Statistiken für alle Beobachtungen
overall_stats = df_latex.describe().drop(index=['count', '25%', '75%'])
overall_stats.rename(index={'50%': 'median'}, inplace=True)

# Erstellung der deskriptiven Statistiken für jede Gruppe
grouped_stats = {
    'overall': overall_stats
}

for treatment, group in df_latex.groupby('treatment', observed=True):
    group_stats = group.describe().drop(index=['count', '25%', '75%'])
    group_stats.rename(index={'50%': 'median'}, inplace=True)
    grouped_stats[treatment] = group_stats

# Zusammenführen der Statistiken in einer Tabelle
combined_stats = pd.concat(grouped_stats, names=['Treatment'])

# LaTeX-Export des kombinierten DataFrames
df_latex_string = combined_stats.to_latex(
    escape=False,
    multirow=True,
    multicolumn=True,
    column_format="l|lrrrrrrrrr", 
    float_format="{:0.2f}".format
)

# LaTeX-Tabellencode erstellen
latex_table = f"""
\\begin{{table}}[h]
    \\centering
    \\caption{{Feldversuch 2 - Ergebnisse, Schwingungsparameter deskriptive Statistiken (Gesamt und gruppiert über Treatment), Amplituden korrigiert über \\texttt{{rope\\_release}}, 9 Beobachtung je Gruppe, jeweils Mittelwert für 4 Elastometer}}
    \\begin{{adjustbox}}{{max width=\\textwidth}}
    {df_latex_string}
    \\end{{adjustbox}}
    \\label{{tab:Feldversuch_2_Deskriptive_Statistiken_Schwingungsparameter}}
\\end{{table}}
"""

print(latex_table)

#### Visualisierung der bereinigten Ergebnisse

Abschließend werden die bereinigten Werte grafisch dargestellt, um die Unterschiede zwischen den Behandlungen in Abwesenheit des `rope_release`-Einflusses zu verdeutlichen. Dies zeigt, wie sich die Treatments auf die Parameter auswirken würden, wenn für alle Einheiten die gleiche mittlere Vorspannung gelten würde.

In [ ]:
relevant_vars_rope_release = ['m_amplitude', 'm_amplitude_2', 'max_strain', 'max_compression']

n_vars = len(relevant_vars_rope_release)
n_cols = 2  # Links Original, rechts angepasst
n_rows = n_vars  # Eine Zeile pro Variable

fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))

for i, var in enumerate(relevant_vars_rope_release):
    # Linke Spalte: Original (grouped_df)
    sns.boxplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,0], x="treatment", y=var, data=grouped_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,0].set_title(f"original: {var}")
    axes[i,0].set_ylabel(var)

    # Rechte Spalte: Angepasst (grouped_adjusted_df)
    sns.boxplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i,1], x="treatment", y=var, data=grouped_adjusted_df, 
                  dodge=False, c="black", jitter=True, size=5)
    axes[i,1].set_title(f"adjusted : {var}")
    axes[i,1].set_ylabel(var)
    
    # Y-Limits von links holen
    y_min, y_max = axes[i,0].get_ylim()
    # Y-Limits auf rechts anwenden
    axes[i,1].set_ylim(y_min, y_max)

plt.tight_layout()
plt.show()

PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_comparison", subdir="combined")


Schritt 2: Durchführung von Post-hoc-Tests

Festzustellen welche paarweisen Unterschiede zwischen den Treatments signifikant sind.

In [ ]:
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Wir gehen davon aus, dass grouped_adjusted_df existiert und die Spalten 'treatment' sowie die Variablen aus 'variables' enthält.

for var in variables_spec:
    # Tukey HSD Test durchführen
    # Annahme: Die Spalte 'treatment' enthält die Gruppennamen z.B. 'free', 'gefa_dynamic', 'cobra_static'
    # pairwise_tukeyhsd benötigt die abhängige Variable und die Gruppen.
    tukey_results = pairwise_tukeyhsd(endog=grouped_adjusted_df[var],
                                      groups=grouped_adjusted_df['treatment'],
                                      alpha=0.05)
    
    print(f"--- Post-Hoc Test (Tukey HSD) für Variable: {var} ---")
    print(tukey_results.summary())
    print("\n")


In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 4 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release", subdir="combined")

In [ ]:
# Anzahl der relevanten Variablen und Layout für die Subplots definieren
n_vars, n_cols, n_rows = len(variables_spec), 2, (len(variables_spec) + 1) // 2

# Subplots erstellen
fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 5 * n_rows))
axes = axes.flatten()

# Für jede relevante Variable einen Plot erstellen
for i, var in enumerate(variables_spec):
    sns.boxplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, palette=treatment_color_dict, hue="treatment", legend=False, dodge=False)
    sns.stripplot(ax=axes[i], x="treatment", y=var, data=grouped_adjusted_df, dodge=False, c="black", jitter=True, size=5)
    axes[i].set_title(f"Einfluss von Treatment auf {var}")
    axes[i].set_ylabel(var)

# Layout anpassen, Plot anzeigen und speichern
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="effect_of_treatment_without_rope_release_spec", subdir="combined")

#### Vorhergesagte Werte extrahieren und Boxplots für die Sensoren erstellen

In [ ]:
# Vorhergesagte Werte aus den Modellen extrahieren
for variable in variables:
    df[f'predicted_{variable}'] = models[variable].fittedvalues

# Boxplots erstellen mit den vorhergesagten Werten
fig, axes = plt.subplots(4, 2, figsize=(8, 8))
axes = axes.flatten()

for i, variable in enumerate(variables):
    sns.boxplot(ax=axes[i], x='treatment', y=f'predicted_{variable}', data=df, palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    axes[i].set_title(f'{variable} by Treatment')
    axes[i].set_xlabel('Treatment')
    axes[i].set_ylabel(f'Predicted {variable}')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig, filename="predicted_effect_for_treatment", subdir="combined")

In [ ]:
def annotate_tukey(ax, tukey_result, significance_level=0.05):
    """
    Fügt eine Textbox mit den Tukey-Test-Ergebnissen und dem festgelegten Signifikanzniveau in den Plot ein.
    
    Parameters:
    ax (matplotlib.axes): Die Achse, auf der der Plot gezeichnet wird.
    tukey_result (TukeyHSDResults): Die Ergebnisse des Tukey HSD Tests.
    significance_level (float): Das Signifikanzniveau, standardmäßig 0.05.
    """
    # Definiere die gewünschte Reihenfolge der Vergleiche
    comparisons_order = [('free', 'gefa_dynamic'), ('free', 'cobra_static'), ('gefa_dynamic', 'cobra_static')]

    # Text für die Annotation zusammenstellen
    text_str = f"Tukey HSD Results: \n(Significance level = {significance_level:.2f})\n\n"
    
    # Durchlaufe die gewünschte Vergleichsreihenfolge
    for group1, group2 in comparisons_order:
        # Filtere die korrekte Paarung aus den Tukey-Ergebnissen
        for i in range(len(tukey_result._results_table.data[1:])):
            pair = tukey_result._results_table.data[i + 1]
            if (pair[0] == group1 and pair[1] == group2) or (pair[0] == group2 and pair[1] == group1):
                p_value = tukey_result.pvalues[i]
                significance = "*" if p_value < significance_level else "n.s."
                text_str += f"\n{group1} vs {group2}: \np = {p_value:.4f} ({significance})\n\n"
    
    # Textbox am Rand des Plots hinzufügen
    ax.annotate(text_str, xy=(1.01, 0.1), xycoords='axes fraction', va='center', ha='left')

# Einzelne Plots für jede Variable erstellen und speichern
for variable in variables:
    fig, ax = plt.subplots(figsize=(8, 4))
    
    # Boxplot für die aktuelle Variable
    sns.boxplot(ax=ax, x='treatment', y=f'predicted_{variable}', data=df, 
                palette=treatment_color_dict, hue='treatment', dodge=False, legend=False)
    
    # Tukey-Test für die aktuelle Variable
    tukey_result = tukey_results[variable]
    
    # Tukey-Ergebnisse annotieren
    annotate_tukey(ax, tukey_result)
    
    ax.set_title(f'{variable} by Treatment')
    ax.set_xlabel('Treatment')
    ax.set_ylabel(f'Predicted {variable}')
    
    plt.tight_layout()
    plt.show()

    # Plot speichern
    plot_filename = f"{variable}_effect_for_treatment"
    PLOT_MANAGER.save_plot(fig, filename=plot_filename, subdir="osc_variables_box")


In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_treatment_describe = (df.drop(['id', 'ptq_sensor_name'], axis=1)
                         .groupby('treatment', observed=True)
                         .describe())

df_treatment_describe = df_treatment_describe.reset_index()
df_treatment_describe.round(2)

In [ ]:
df.dtypes

In [ ]:
# Gruppieren des DataFrames nach 'treatment' und Entfernen unnötiger Spalten
df_sensor = (df.drop(['id', 'release_force_target', 'ls3_rope_release', 'ls3_cable_max', 'location', 'height', 'diameter', 'direction'], axis=1).
             groupby(['treatment', 'ptq_sensor_name'], observed=True).
             mean())  #.T
#df_sensor.round(4)

In [ ]:
# Gruppieren des DataFrames und Anwenden von mean für ptq_sensor_name 
df_id = ((df.drop(['ptq_sensor_name', 'location', 'height', 'diameter', 'direction'], axis=1)
          .groupby(['treatment', 'id'], observed=True)
          .mean())
         .reset_index())

df_id.round(4).head()

### Zusammenhangsanalyse für LS3 und PTQ

In [ ]:
# Auswahl der neuen Spaltennamen für die Korrelationsmatrix
columns_corr = ['ptq_m_amplitude',
                'ptq_m_amplitude_2',
                'ptq_initial_amplitude',
                'ptq_damping_coeff',
                'ptq_angular_frequency',
                'ptq_y_shift',
                'ptq_pearson_r',
                #'ptq_nrmse',
                'ptq_nmae',
                'release_force_target',
                'ls3_rope_release',
                'ls3_cable_max']
df_corr = df_id.copy()[columns_corr]

# Berechnung der Korrelationsmatrix
correlation_matrix = df_corr.corr()

# Visualisierung der Korrelationsmatrix mit Seaborn
fig1, ax = plt.subplots(figsize=(8, 8))  # Anpassen der Größe der Grafik
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f', ax=ax, annot_kws={'size': 10})

# Titel und Schriftgrößen anpassen
#plt.title('Correlation Matrix for LS3 and PTQ', fontsize=18)
ax.set_xticklabels(ax.get_xmajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_yticklabels(ax.get_ymajorticklabels(), fontsize=12, rotation=45, ha='right')
ax.set_xlabel('')
ax.set_ylabel('')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig1, filename="correlation_matrix", subdir="combined")

In [ ]:
# ANOVA für 'ls3_rope_release'
model_rope_release = smf.ols('ls3_rope_release ~ treatment', data=df_id).fit()
anova_rope_release = sm.stats.anova_lm(model_rope_release, typ=2)

# ANOVA für 'ls3_cable_max'
model_cable_max = smf.ols('ls3_cable_max ~ treatment', data=df_id).fit()
anova_cable_max = sm.stats.anova_lm(model_cable_max, typ=2)

# Zusammenfassungen der Modelle
summary_rope_release = model_rope_release.summary()
summary_cable_max = model_cable_max.summary()
# Zusammenfassungen der Modelle
summary_rope_release_latex = model_rope_release.summary().as_latex()
summary_cable_max_latex = model_cable_max.summary().as_latex()

#print(summary_rope_release_latex)
#print(summary_cable_max_latex)

anova_rope_release, summary_rope_release, anova_cable_max, summary_cable_max

#### Zusammenhang Vorspannung und resultierende Lastspitzen

In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude", subdir="combined")


In [ ]:
fig3, ax1 = plt.subplots(figsize=(8, 5))
ax2 = ax1.twinx()
ax2.axis('off')
y_pos_init = 0.1
for idx, (treatment, color) in enumerate(treatment_color_dict.items()):
    subset = df_id[df_id['treatment'] == treatment]
    if subset['ptq_m_amplitude_2'].isna().all():
        continue
    sns.regplot(x='ls3_rope_release', y='ptq_m_amplitude_2', data=subset, ax=ax1, color=color, label=treatment,
                ci=95)
    stats_text = annotate_stats(subset['ls3_rope_release'], subset['ptq_m_amplitude_2'])
    ax2.annotate(f"{treatment}:\n{stats_text}", xy=(1.01, y_pos_init + idx * 0.3), xycoords='axes fraction')
#ax1.set_title('Correlation Between Release Force and Elongation Amplitude 2')
ax1.set_xlabel('Release Force [kN]')
ax1.set_ylabel('Elongation Amplitude 2 [$\mu$m] (mean for all Sensors)')
ax1.legend(title='Treatment', loc='upper left')
plt.tight_layout()
plt.show()
PLOT_MANAGER.save_plot(fig3, filename=f"ls3_release_force_vs_ptq_m_amplitude_2", subdir="combined")

In [ ]:
# Funktion zur Durchführung des ANOVA-Tests und Berechnung der Effektstärke (Eta Squared)
def perform_anova_and_effect_size(df: pd.DataFrame, variable: str, treatments: List[str]) -> str:
    groups = [df[df['treatment'] == treatment][variable].dropna() for treatment in treatments]
    f_stat, p_value = f_oneway(*groups)

    # Berechnung der Effektstärke (Eta Squared)
    n = sum([len(g) for g in groups])
    ss_total = sum([(x - df[variable].mean()) ** 2 for g in groups for x in g])
    eta_squared = f_stat * len(groups) / (f_stat * len(groups) + (n - len(groups)))

    # Überprüfung der Signifikanz
    significance = "*" if p_value < 0.05 else ""

    return f"{variable}: {significance}\nF-statistic = {f_stat:.2f}\np-value = {p_value:.2e}\nEta Squared = {eta_squared:.2f}"

In [ ]:
# Funktion zur Erstellung von Boxplots
def create_boxplot(df: pd.DataFrame, variable: str, group_by: str, ax: plt.Axes, color_dict: Dict[str, str], perform_stats: bool) -> None:
    valid_df = df.dropna(subset=[variable])
    sns.boxplot(x=group_by, y=variable, hue=group_by, data=valid_df, ax=ax, palette=color_dict, dodge=False)
    ax2 = ax.twinx()
    ax2.axis('off')
    if perform_stats:
        stats_text = perform_anova_and_effect_size(valid_df, variable, valid_df[group_by].unique())
        ax2.annotate(stats_text, xy=(1.01, 0.1), xycoords='axes fraction')
    ax.set_title(f'Einfluss von {group_by} auf {variable}')
    ax.set_xlabel(group_by)
    ax.set_ylabel(variable)

# Funktion zur Erstellung kombinierter Plots
def create_combined_plot(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], num_columns: int = 3, perform_stats: bool = False) -> None:
    num_rows = len(columns) // num_columns + (len(columns) % num_columns > 0)
    fig, axes = plt.subplots(num_rows, num_columns, figsize=(16, 4 * num_rows))
    axes = axes.flatten()

    for idx, variable in enumerate(columns):
        create_boxplot(df, variable, group_by, axes[idx], color_dict, perform_stats)

    plt.tight_layout()
    plt.show()
    PLOT_MANAGER.save_plot(fig, filename=f"combined_plot_{group_by}", subdir="combined")

# Funktion zur Erstellung einzelner Plots
def create_individual_plots(df: pd.DataFrame, columns: List[str], group_by: str, color_dict: Dict[str, str], perform_stats: bool = False) -> None:
    for variable in columns:
        fig, ax = plt.subplots(figsize=(8, 5))
        create_boxplot(df, variable, group_by, ax, color_dict, perform_stats)
        plt.tight_layout()
        #plt.show()
        PLOT_MANAGER.save_plot(fig, filename=f"{group_by}_{variable}", subdir="individual_plots")

In [ ]:
columns = ['ptq_m_amplitude',
           'ptq_m_amplitude_2',
           'ptq_initial_amplitude',
           'ptq_damping_coeff',
           'ptq_angular_frequency',
           'ptq_y_shift',
           'ptq_pearson_r',
           #'ptq_nrmse',
           #'ptq_nmae',
           #'release_force_target',
           'ls3_rope_release',
           'ls3_cable_max'
           ]

# Beispiel: Erstellen von Plots gruppiert nach 'treatment'
create_combined_plot(df, columns, 'treatment', treatment_color_dict, perform_stats=True)
create_individual_plots(df, columns, 'treatment', treatment_color_dict, perform_stats=True)

In [ ]:
# Beispiel: Erstellen von Plots gruppiert nach 'ptq_sensor_name'
columns = ['ptq_m_amplitude', 'ptq_m_amplitude_2', 'ptq_initial_amplitude', 'ptq_damping_coeff', 'ptq_angular_frequency', 'ptq_pearson_r']

create_combined_plot(df, columns, 'ptq_sensor_name', sensor_color_dict)
create_individual_plots(df, columns, 'ptq_sensor_name', sensor_color_dict)